In [ ]:
# TODO: move this to an appropriate folder
import inspect
import dill
import heapq # only used for second step of code modification with bindings tracking and in exitlines func

# Notebook Use

In [ ]:
debug_messages = True

# Frame definitions and init

In [2]:
class Frame():
    # I, Noemi, refer to this as a FrameNode to remove ambiguity with FrameObjects for the time being. 
    is_global = False
    exitline = None

    def __init__(self, name = None, bindings=None, parent=None, fobj=None):
        self.parent = parent
        self.children = set()
        if bindings is None:
            self.bindings = {}
        self.name = name
        # self's frame object
        self.fobj = fobj

    def bind(self, name, value):
        self.bindings[name] = value # Note: this will allow for duplicate entries. need to figure out a better way to do this. 
    
    def bind(self, name_value_dict):
        self.bindings = name_value_dict # Note: this will allow for duplicate entries. need to figure out a better way to do this. 
    
    def set_name(self, name):
        self.name = name
    
    def add_child(self, child):
        self.children.add(child)
    
    def set_exitline(self, exitline):
        self.exitline = exitline

In [3]:
# a dictionary containing frames with associated variables in the form of a tuple: (type, name, mem loc, value)
envframes_dict = {"global":{}}
fobj_name_dict = {}
# associates a frame object to a frame node (or in the case of global, at least for the time being, "global" to a frame node.)
fobj_framenode_dict = {}
env_mutables = {}
cnter = 1

# TODO: maybe name "___main___" instead of global? hmmmmmm but if we have wrapper func then no. 
def add_new_frame(name = None, bindings = None, frame_dict = envframes_dict): # func should always refer to the caller
    curr_frame = inspect.currentframe().f_back
    if bindings is None:
        bindings = {}
    # if no frame nodes have been initialized, set the frame as the global frame and the root
    if len(fobj_framenode_dict.keys()) == 0:
        name = "global"
        fobj_framenode_dict[name] = Frame(bindings = bindings, fobj = curr_frame) # TODO: add fobj
        # OLD: TODO: also current_fobj not listed, fix
        # init FrameNode
        frame = Frame(name = name, bindings = bindings, fobj = curr_frame)
        envframes_dict[name] = {"name": None, "parent": None, "parent_fobj": None, "curr_fobj": curr_frame, "bindings":bindings}

    else: 
        name = curr_frame.f_code.co_name
        f_name = "f" + name #temp TODO: add number
        parent_frame = curr_frame.f_back
        # TODO: why global be stupid? fix.
        parent = fobj_framenode_dict[parent_frame] if parent_frame in fobj_framenode_dict else fobj_framenode_dict["global"] # frame_name_dict[parent_frame] if parent_frame in frame_name_dict else "global"
        
        # OLD: dictionary rep of frame associations. Use for debugging only. 
        envframes_dict[f_name] = {"name": name, "parent": parent, "parent_fobj": parent_frame, "curr_fobj": curr_frame, "bindings":bindings}
        # init FrameNode
        frame = Frame(name = name, bindings = bindings, parent = parent, fobj=curr_frame)
        # add self to the pairing dict
        fobj_framenode_dict[curr_frame] = frame
        # modify parent to include self as a child
        parent.add_child(fobj_framenode_dict[curr_frame])
        # OLD: dictionary rep between fobj and frame
        fobj_name_dict[curr_frame] = name
    return frame, fobj_framenode_dict, envframes_dict, fobj_name_dict # messy

# code live insertion

### step 1: initialize FrameNodes and tracing line exits

#### actual

In [55]:
example_meow = """
# TEST: meow
def glob_meow():
    print("in global")
    def A_meow():

        # this is a test message
        z = 5
        return z
    A_meow()

glob_meow()"""

In [29]:
example_meow2 = """
# TEST: meow2
def glob_meow2():
    print("in global")
    def A_meow2():

        # this is a test comment
        z = 5
        if True:
            return z

            # test comment
        else:
            return z - 5

    A_meow2()
    
glob_meow2()"""

insert FrameNode tracking into the code

In [30]:
# TODO: do this for lambda as well
# env_mutables
def insertFrameTracking(code):
    # TODO: maybe make it so you can input a func directly
    """ code: code as list or string. 
    this function adds a new line of code initializing the FrameNode in each function definition. """
    # does this by finding a 'def' statement, then inserting the init immediately after
    # if code is given as a string
    if isinstance(code, str):
        code_list = code.split("\n")
    #elif callable(code):
    #    code_list = inspect.getsourcelines(code)[0]
    elif isinstance(code, list):
        code_list = code
    else:
        raise Exception("inputted code not proper format.")

    # TODO: add mutables
    # first init all of the vars
    code_list.insert(0, "envframes_dict = {'global':{}}")
    code_list.insert(1, "fobj_framenode_dict = {}")
    code_list.insert(2, "fobj_name_dict = {}")

    #  loop through all remaining code.
    i = 3 # modify to consider initalized vars
    while i < len(code_list):
        if code_list[i].lstrip(' ')[:4] == "def ":
            def_line = i
            # find the indent size after the def
            whitespace = 0
            while whitespace == 0:
                i = i + 1
                # check to make sure it's not just an empty line
                if len(code_list[i].split()) > 0:
                    # get size of whitespace before code
                    whitespace = len(code_list[i]) - len(code_list[i].lstrip(' '))
                    # set this back to the previous line just in case the first non-empty line is a def statement.
                    i = i - 1
            # insert the code to create the FrameNode
            # TODO: make sure none of the variables are already in scope in the code.
            code_list.insert(def_line + 1, ' '*whitespace + "frame, fobj_framenode_dict, envframes_dict, frame_name_dict = add_new_frame() # messy")
        i = i + 1
    # if something was in a define statement, handle that.
    return str.join("\n", code_list)

# TODO: maybe do this in function?
# get modified code with Frame initialization
new_code = insertFrameTracking(example_meow2)
# THEN RUN EXEC(newcode), 
exec(new_code)

in global


End-line tracing

In [31]:
# END-LINE TRACING
def getexitlines():
    # TODO: might get rid of: edits all FrameNodes to include a reference to which line it ends on (in the modified code)
    """edits all FrameNodes to include a reference to which line it ends on in .exitline. 
    Additionally returns a max heap of exit lines, and a dictionary mapping exit lines to Frame objects."""
    if debug_messages: print("========== RUNNING getexitlines ==========") # TODO: make debug_messages a decorator and apply it to all funcs?

    exitlines_pq = []
    exitlines_fobj_dict = {} # might be able to replace this with an exitlines to var list mapping. if so, change description of this func. 
    for fobj in fobj_framenode_dict:
        frame = fobj_framenode_dict[fobj]
        if fobj == "global":
            fobj = frame.fobj
        exitline = frame.fobj.f_lineno - 1
        exitlines_pq.append(exitline)
        exitlines_fobj_dict[exitline] = fobj
        frame.set_exitline(exitline)

    if debug_messages: print("========== COMPLETED getexitlines ==========")
    heapq._heapify_max(exitlines_pq)
    return exitlines_pq, exitlines_fobj_dict
exitlines, exitlines_fobj_dict = getexitlines()

========== RUNNING getexitlines ==========
========== COMPLETED getexitlines ==========


In [189]:
# TEST BLOCK
fobj_framenode_dict[envframes_dict["fA_meow2"]["curr_fobj"]]
frame_test = envframes_dict["fA_meow2"]["curr_fobj"]
frame_test.f_code.co_varnames

('fobj_framenode_dict', 'env_frames', 'frame_name_dict', 'z')

### step 2: creating frame objects

In [37]:
def insertBindingTracking(exitlines, code):
    """this function inserts tracking for bindings in each function just before it would otherwise end."""
    # keeps track of how many lines have been inserted above the 
    # if code is given as a string
    exitlines = exitlines.copy()
    code_list = code.split("\n")
    
    # get largest exitline
    while len(exitlines) > 0:
        # get the current exitline (also the first line we insert into)!
        exitline = heapq._heappop_max(exitlines)
        i = exitline
        whitespace = len(code_list[i]) - len(code_list[i].lstrip(' '))

        # set variable names for things we are using to store tracking info.
        trk_var_names = ["returnval", "locs", "bindings", "fobj_framenode_dict", "envframes_dict", "frame_name_dict"]
        # if any of the initial variable names are in the local variable scope, we need to change the offending name(s). TODO: check if this is true once impl is done?
        for k in range(len(trk_var_names)):
            while trk_var_names[k] in exitlines_fobj_dict[exitline].f_code.co_varnames:
                trk_var_names[k] = str(hash(trk_var_names[k] + str(exitline)))[:7]

        if code_list[i].lstrip(' ')[:7] == "return ":
            returnval = code_list[i].lstrip(' ')[7:]
            # set name for returnval in the code
            # change the return line to return the stored value
            code_list[i] = ' '*whitespace + "return " + trk_var_names[0]
            # insert a line that records the return value
            code_list.insert(i, ' '*whitespace + trk_var_names[0] + '=' + returnval)
            i = i + 1
        # store local variables at this point
        code_list.insert(i, ' '*whitespace + trk_var_names[1] + '=' "locals()")
        # store bindings at this point
        # TODO: 'var' might also be an issue here..
        code_list.insert(i + 1, ' '*whitespace + trk_var_names[2] + '=' "{var:(type(" + trk_var_names[1] + "[var]), " + trk_var_names[1] + "[var]) for var in " + trk_var_names[1] + "}")
        # TODO: var names
        code_list.insert(i + 2, ' '*whitespace + "frame.bind(" + trk_var_names[2] + ")")

    return str.join("\n", code_list)

# get modified code with Frame initialization
newnew_code = insertBindingTracking(exitlines, new_code)
# THEN RUN EXEC(newcode), 
exec(new_code)

in global


# re-organizing of code into a class

In [38]:
class FrameTree():
    originalcodestring = None
    codestring = None
    root = None
    # TODO: if i set these to {}, does that make these static?
    envframes_dict = None
    fobj_name_dict = None
    env_mutables = None

    def __init__(self, codestring):
        self.originalcodestring = codestring
        self.codestring = codestring
        # a dictionary containing frames with associated variables in the form of a tuple: (type, name, mem loc, value)
        self.envframes_dict = {"global":{}}
        self.fobj_name_dict = {}
        # associates a frame object to a frame node (or in the case of global, at least for the time being, "global" to a frame node.)
        self.fobj_framenode_dict = {}
        self.env_mutables = {}
    
    def add_new_frame(self, name = None, bindings = None): # func should always refer to the caller
        curr_frame = inspect.currentframe().f_back
        if bindings is None:
            bindings = {}
        # if no frame nodes have been initialized, set the frame as the global frame and the root
        if len(fobj_framenode_dict.keys()) == 0:
            name = "global"
            self.fobj_framenode_dict[name] = Frame(bindings = bindings, fobj = curr_frame) # TODO: add fobj
            # OLD: TODO: also current_fobj not listed, fi          # init FrameNode
            self.frame = Frame(name = name, bindings = bindings, fobj = curr_frame)
            self.envframes_dict[name] = {"name": None, "parent": None, "parent_fobj": None, "curr_fobj": curr_frame, "bindings":bindings}

        else: 
            name = curr_frame.f_code.co_name
            f_name = "f" + name #temp TODO: add number
            parent_frame = curr_frame.f_back
            # TODO: why global be stupid? fix.
            parent = fobj_framenode_dict[parent_frame] if parent_frame in fobj_framenode_dict else fobj_framenode_dict["global"] # frame_name_dict[parent_frame] if parent_frame in frame_name_dict else "global"
            
            # OLD: dictionary rep of frame associations. Use for debugging only. 
            self.envframes_dict[f_name] = {"name": name, "parent": parent, "parent_fobj": parent_frame, "curr_fobj": curr_frame, "bindings":bindings}
            # init FrameNode
            self.frame = Frame(name = name, bindings = bindings, parent = parent, fobj=curr_frame)
            # add self to the pairing dict
            self.fobj_framenode_dict[curr_frame] = frame
            # modify parent to include self as a child
            parent.add_child(fobj_framenode_dict[curr_frame])
            # OLD: dictionary rep between fobj and frame
            self.fobj_name_dict[curr_frame] = name
    
    # TODO: do this for lambda as well
    def insertFrameTracking(self, codestring):
        # TODO: maybe make it so you can input a func directly
        """ codestring: code as a string. 
        This function adds a new line of code initializing the FrameNode in each function definition. 
        It then runs the new code made to get exitlines."""
        # does this by finding a 'def' statement, then inserting the init immediately after
        # if code is given as a string
        code_list = codestring.split("\n")

        # TODO: add mutables
        # first init all of the vars
        code_list.insert(0, "envframes_dict = {'global':{}}")
        code_list.insert(1, "fobj_framenode_dict = {}")
        code_list.insert(2, "fobj_name_dict = {}")

        #  loop through all remaining code.
        i = 3 # modify to consider initalized vars
        while i < len(code_list):
            if code_list[i].lstrip(' ')[:4] == "def ":
                def_line = i
                # find the indent size after the def
                whitespace = 0
                while whitespace == 0:
                    i = i + 1
                    # check to make sure it's not just an empty line
                    if len(code_list[i].split()) > 0:
                        # get size of whitespace before code
                        whitespace = len(code_list[i]) - len(code_list[i].lstrip(' '))
                        # set this back to the previous line just in case the first non-empty line is a def statement.
                        i = i - 1
                # insert the code to create the FrameNode
                # TODO: make sure none of the variables are already in scope in the code.
                code_list.insert(def_line + 1, ' '*whitespace + "frame, fobj_framenode_dict, envframes_dict, frame_name_dict = add_new_frame() # messy")
            i = i + 1

        # get modified code with Frame initialization
        new_code = insertFrameTracking(example_meow2)
        # THEN RUN EXEC(newcode), 
        exec(new_code)
        self.codestring = str.join("\n", code_list)
        

# Trash